In [23]:
import maliput

from pprint import pprint

In [2]:
help(maliput.api)

Help on module maliput.api in maliput:

NAME
    maliput.api

SUBMODULES
    rules

CLASSES
    pybind11_builtins.pybind11_object(builtins.object)
        BranchPoint
        BranchPointId
        HBounds
        InertialPosition
        Intersection
        IntersectionBook
        IsoLaneVelocity
        Junction
        JunctionId
        Lane
        LaneEnd
        LaneEndSet
        LaneId
        LanePosition
        LanePositionResult
        LaneSRange
        LaneSRoute
        RBounds
        RoadGeometry
        RoadGeometry.IdIndex
        RoadGeometryId
        RoadNetwork
        RoadPosition
        RoadPositionResult
        Rotation
        SRange
        Segment
        SegmentId
        UniqueId
        Which
    
    class BranchPoint(pybind11_builtins.pybind11_object)
     |  Method resolution order:
     |      BranchPoint
     |      pybind11_builtins.pybind11_object
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  GetASide(...

In [3]:
help(maliput.math)

Help on module maliput.math in maliput:

NAME
    maliput.math

CLASSES
    pybind11_builtins.pybind11_object(builtins.object)
        Quaternion
        RollPitchYaw
        Vector3
        Vector4
    
    class Quaternion(pybind11_builtins.pybind11_object)
     |  Method resolution order:
     |      Quaternion
     |      pybind11_builtins.pybind11_object
     |      builtins.object
     |  
     |  Methods defined here:
     |  
     |  __init__(...)
     |      __init__(self: maliput.math.Quaternion, arg0: float, arg1: float, arg2: float, arg3: float) -> None
     |  
     |  __str__(...)
     |      __str__(self: maliput.math.Quaternion) -> str
     |  
     |  coeffs(...)
     |      coeffs(self: maliput.math.Quaternion) -> maliput.math.Vector4
     |  
     |  w(...)
     |      w(self: maliput.math.Quaternion) -> float
     |  
     |  x(...)
     |      x(self: maliput.math.Quaternion) -> float
     |  
     |  y(...)
     |      y(self: maliput.math.Quaternion) -> float
   

In [4]:
manager = maliput.plugin.MaliputPluginManager()
manager.ListPlugins()


['maliput_dragway', 'maliput_multilane', 'maliput_malidrive']

In [6]:
# https://shiny-fiesta-6b790eb7.pages.github.io/from_doxygen/html/deps/maliput_malidrive/html/group__road__network__configuration__builder__keys.html
# 
road_network_options = {
    "opendrive_file": "../src/maliput_malidrive/maliput_malidrive/resources/figure8_trafficlights/figure8_trafficlights.xodr"
}


In [7]:
network1 = maliput.plugin.create_road_network_from_plugin("maliput_malidrive", road_network_options)

In [8]:
help(network1.road_geometry())
help(network1.road_geometry().ById())
help(network1.road_geometry().junction(0))
help(network1.road_geometry().junction(0).segment(0))
help(network1.road_geometry().junction(0).segment(0).lane(0))
help(network1.road_geometry().junction(0).segment(0).lane(0).id())
help(maliput.api.SRange)
help(maliput.api.LaneSRange)
geometry = network1.road_geometry()

Help on RoadGeometry in module maliput.api object:

class RoadGeometry(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      RoadGeometry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  ById(...)
 |      ById(self: maliput.api.RoadGeometry) -> maliput::api::RoadGeometry::IdIndex
 |  
 |  FindRoadPositions(...)
 |      FindRoadPositions(self: maliput.api.RoadGeometry, inertial_position: maliput.api.InertialPosition, radius: float) -> List[maliput.api.RoadPositionResult]
 |  
 |  ToRoadPosition(...)
 |      ToRoadPosition(self: maliput.api.RoadGeometry, inertial_position: maliput.api.InertialPosition) -> maliput.api.RoadPositionResult
 |  
 |  ToRoadPositionByHint(...)
 |      ToRoadPositionByHint(self: maliput.api.RoadGeometry, inertial_position: maliput.api.InertialPosition, hint: maliput.api.RoadPosition) -> maliput.api.RoadPositionResult
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  Se

In [9]:
# Iterate all items.
print("Geometry. Num junctions: {}".format(geometry.num_junctions()))
for junction_index in range(geometry.num_junctions()):
    junction = geometry.junction(junction_index)
    # help(junction)
    print("  Junction {}: Segments: {}".format(junction.id().string(), junction.num_segments()))
    for segment_index in range(junction.num_segments()):
        segment = junction.segment(segment_index)
        print("    Segment {}: Lanes: {}".format(segment.id().string(), segment.num_lanes()))
        for lane_index in range(segment.num_lanes()):
            lane = segment.lane(lane_index)
            print("    Lane {}: length: {}".format(lane.id().string(), lane.length()))
            

Geometry. Num junctions: 11
  Junction 0_0: Segments: 1
    Segment 0_0: Lanes: 2
    Lane 0_0_-1: length: 174.77007946851094
    Lane 0_0_1: length: 169.27190949567478
  Junction 1_0: Segments: 1
    Segment 1_0: Lanes: 2
    Lane 1_0_-1: length: 90.20944440138805
    Lane 1_0_1: length: 90.23055559858
  Junction 10: Segments: 84
    Segment 18_0: Lanes: 1
    Lane 18_0_1: length: 0.1799999999981746
    Segment 18_1: Lanes: 1
    Lane 18_1_1: length: 2.100000000000009
    Segment 18_2: Lanes: 1
    Lane 18_2_1: length: 0.05000000000001136
    Segment 18_3: Lanes: 1
    Lane 18_3_1: length: 13.529999999999967
    Segment 18_4: Lanes: 1
    Lane 18_4_1: length: 0.05000000000001136
    Segment 18_5: Lanes: 1
    Lane 18_5_1: length: 2.090000000000001
    Segment 18_6: Lanes: 1
    Lane 18_6_1: length: 0.17999999999818642
    Segment 19_0: Lanes: 1
    Lane 19_0_-1: length: 0.1799999999981746
    Segment 19_1: Lanes: 1
    Lane 19_1_-1: length: 2.100000000000009
    Segment 19_2: Lanes: 1

In [10]:
# Sample so I can get help.
p_inertial = maliput.api.InertialPosition(0,0,0)
road_position_result = geometry.ToRoadPosition(p_inertial)
road_position = road_position_result.road_position
help(p_inertial)
help(road_position_result)
help(road_position)
help(road_position.lane)
help(road_position.pos)


Help on InertialPosition in module maliput.api object:

class InertialPosition(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      InertialPosition
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Distance(...)
 |      Distance(self: maliput.api.InertialPosition, inertial_position: maliput.api.InertialPosition) -> float
 |  
 |  __eq__(...)
 |      __eq__(self: maliput.api.InertialPosition, arg0: maliput.api.InertialPosition) -> bool
 |  
 |  __init__(...)
 |      __init__(self: maliput.api.InertialPosition, x: float, y: float, z: float) -> None
 |  
 |  length(...)
 |      length(self: maliput.api.InertialPosition) -> float
 |  
 |  set_x(...)
 |      set_x(self: maliput.api.InertialPosition, arg0: float) -> None
 |  
 |  set_y(...)
 |      set_y(self: maliput.api.InertialPosition, arg0: float) -> None
 |  
 |  set_z(...)
 |      set_z(self: maliput.api.InertialPosition, arg0: float) -> None
 |  
 |  x(...)
 

In [11]:
# Perform some to road queries, to lane queries, and back.
positions = [
    [0, 0, 0],
    [0, 1, 0],
    [0, 25, 0]
]
for position in positions:
    p_inertial = maliput.api.InertialPosition(*position)
    # Would be nice to have __repr__
    print("inertial {}, {}, {}".format(p_inertial.x(), p_inertial.y(), p_inertial.z()))
    road_position_result = geometry.ToRoadPosition(p_inertial)
    road_position = road_position_result.road_position
    print(
        "road position {}, ({},{},{})".format(
            str(road_position.lane.id().string()),
            road_position.pos.s(),
            road_position.pos.r(),
            road_position.pos.h()
        )
    )
    # back to inertial
    p_roundtrip = road_position.lane.ToInertialPosition(road_position.pos)
    print("inertial roundtrip {}, {}, {}".format(p_roundtrip.x(), p_roundtrip.y(), p_roundtrip.z()))
    
    
    print()

inertial 0.0, 0.0, 0.0
road position 96_3_-1, (6.342075680086242,1.2679070179605731,0.0)
inertial roundtrip 1.6826817716975029e-15, -1.1102230246251565e-16, 0.0

inertial 0.0, 1.0, 0.0
road position 96_3_-1, (6.350642120338619,0.267943710583046,0.0)
inertial roundtrip -9.679756995950584e-16, 1.0, 0.0

inertial 0.0, 25.0, 0.0
road position 2_0_-1, (16.39078594979164,1.2830799963633912,0.0)
inertial roundtrip 5.551115123125783e-17, 25.000000000000004, 0.0



In [12]:
the_range = maliput.api.SRange(0,1)
the_lane_range = maliput.api.LaneSRange(road_position.lane.id(), the_range)
the_range_2 = maliput.api.SRange(1,0.5)
arbitrary_lane_id = network1.road_geometry().junction(0).segment(0).lane(0).id()
the_lane_range_2 = maliput.api.LaneSRange(arbitrary_lane_id, the_range_2)

In [13]:
print("range", the_range.s0(), the_range.s1(), the_range.size(), "with_s", the_range.WithS())
print("range2", the_range_2.s0(), the_range_2.s1(), the_range_2.size(), "with_s", the_range_2.WithS())

range 0.0 1.0 1.0 with_s True
range2 1.0 0.5 0.5 with_s False


In [14]:
overlap1 = the_range.GetIntersection(the_range_2, 0.01)
overlap2 = the_range_2.GetIntersection(the_range, 0.01)
# Order not preserved. Okay with me!
print("intersects", the_range.Intersects(the_range_2, 0.01), "overlap", overlap1.s0(), overlap1.s1(), overlap1.size())
print("intersects", the_range_2.Intersects(the_range, 0.01), "overlap", overlap2.s0(), overlap2.s1(), overlap2.size())
print("lane intersects?", the_lane_range.Intersects(the_lane_range_2, 0.01))

intersects True overlap 0.5 1.0 0.5
intersects True overlap 0.5 1.0 0.5
lane intersects? False


In [16]:
# Now load a road network with rulebooks.
# https://shiny-fiesta-6b790eb7.pages.github.io/from_doxygen/html/deps/maliput_malidrive/html/group__road__network__configuration__builder__keys.html
# 
road_network_options2 = {
    "opendrive_file": "../src/maliput_malidrive/maliput_malidrive/resources/figure8_trafficlights/figure8_trafficlights.xodr",
    "road_rule_book": "../src/maliput_malidrive/maliput_malidrive/resources/figure8_trafficlights/figure8_trafficlights.yaml",
}
road_network_options2["traffic_light_book"] = road_network_options2["road_rule_book"]
road_network_options2["phase_ring_book"] = road_network_options2["road_rule_book"]
road_network_options2["intersection_book"] = road_network_options2["road_rule_book"]


In [17]:
network2 = maliput.plugin.create_road_network_from_plugin("maliput_malidrive", road_network_options2)

In [18]:
help(network2)

Help on RoadNetwork in module maliput.api object:

class RoadNetwork(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      RoadNetwork
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  Contains(...)
 |      Contains(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. Contains(self: maliput.api.RoadNetwork, road_position: maliput.api.RoadPosition) -> bool
 |      
 |      2. Contains(self: maliput.api.RoadNetwork, lane_id: maliput::api::TypeSpecificIdentifier<maliput::api::Lane>) -> bool
 |  
 |  __init__(...)
 |      __init__(self: maliput.api.RoadNetwork, road_geometry: maliput::api::RoadGeometry, rulebook: maliput::api::rules::RoadRulebook, traffic_light_book: maliput::api::rules::TrafficLightBook, intersection_book: maliput::api::IntersectionBook, phase_ring_book: maliput::api::rules::PhaseRingBook, right_of_way_rule_state_provider: maliput::api::rules::RightOfWayRuleStateProvider, phase_provider

In [20]:
registry = network2.rule_registry()
help(registry)

Help on RuleRegistry in module maliput.api.rules object:

class RuleRegistry(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      RuleRegistry
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  BuildDiscreteValueRule(...)
 |      BuildDiscreteValueRule(self: maliput.api.rules.RuleRegistry, id: maliput.api.rules.Rule.Id, type_id: maliput.api.rules.Rule.TypeId, zone: maliput.api.LaneSRoute, values: List[maliput.api.rules.DiscreteValueRule.DiscreteValue]) -> maliput.api.rules.DiscreteValueRule
 |  
 |  BuildRangeValueRule(...)
 |      BuildRangeValueRule(self: maliput.api.rules.RuleRegistry, id: maliput.api.rules.Rule.Id, type_id: maliput.api.rules.Rule.TypeId, zone: maliput.api.LaneSRoute, ranges: List[maliput.api.rules.RangeValueRule.Range]) -> maliput.api.rules.RangeValueRule
 |  
 |  DiscreteValueRuleTypes(...)
 |      DiscreteValueRuleTypes(self: maliput.api.rules.RuleRegistry) -> Dict[maliput.api.rules.Rule.Ty

In [28]:
rule_types = registry.DiscreteValueRuleTypes()
pprint(rule_types)
direction_usage = list(rule_types.keys())[0]
help(direction_usage)

{Direction Usage Rule Type: [<maliput.api.rules.DiscreteValueRule.DiscreteValue object at 0x7f6a1050adf0>,
 Vehicle Exclusive Rule Type: [<maliput.api.rules.DiscreteValueRule.DiscreteValue object at 0x7f6a104e3f30>,
 Vehicle Usage Rule Type: [<maliput.api.rules.DiscreteValueRule.DiscreteValue object at 0x7f6a105055f0>,
                           <maliput.api.rules.DiscreteValueRule.DiscreteValue object at 0x7f6a10505430>]}
Help on TypeId in module maliput.api.rules object:

class TypeId(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      TypeId
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(...)
 |      __eq__(self: maliput.api.rules.Rule.TypeId, arg0: maliput.api.rules.Rule.TypeId) -> bool
 |  
 |  __hash__(...)
 |      __hash__(self: maliput.api.rules.Rule.TypeId) -> int
 |  
 |  __init__(...)
 |      __init__(self: maliput.api.rules.Rule.TypeId, arg0: str) -> None
 |  
 |  __repr__(...)
 |      __r

In [31]:
range_rule_types = registry.RangeValueRuleTypes()
pprint(range_rule_types)
speed_limit = list(range_rule_types.keys())[0]

{Speed Limit Rule Type: [<maliput.api.rules.RangeValueRule.Range object at 0x7f6a10505170>,
                         <maliput.api.rules.RangeValueRule.Range object at 0x7f6a000f0bf0>]}


In [38]:
# Let us try to get states for a lane. 
p_inertial = maliput.api.InertialPosition(5,0,0)
p_road_result = network2.road_geometry().ToRoadPosition(p_inertial)
p_road = p_road_result.road_position

In [41]:
rulebook = network2.rulebook()
help(rulebook)

Help on RoadRulebook in module maliput.api.rules object:

class RoadRulebook(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      RoadRulebook
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  FindRules(...)
 |      FindRules(self: maliput.api.rules.RoadRulebook, ranges: List[maliput.api.LaneSRange], tolerance: float) -> maliput::api::rules::RoadRulebook::QueryResults
 |  
 |  GetDiscreteValueRule(...)
 |      GetDiscreteValueRule(self: maliput.api.rules.RoadRulebook, id: maliput.api.rules.Rule.Id) -> maliput.api.rules.DiscreteValueRule
 |  
 |  GetRangeValueRule(...)
 |      GetRangeValueRule(self: maliput.api.rules.RoadRulebook, id: maliput.api.rules.Rule.Id) -> maliput.api.rules.RangeValueRule
 |  
 |  GetRule(...)
 |      GetRule(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. GetRule(self: maliput.api.rules.RoadRulebook, id: maliput.api.rules.RightOfWayRule.Id) -> maliput.api.rules.RightOf

In [64]:
discrete_provider = network2.discrete_value_rule_state_provider()
help(discrete_provider)
range_provider = network2.range_value_rule_state_provider()
help(range_provider)

Help on DiscreteValueRuleStateProvider in module maliput.api.rules object:

class DiscreteValueRuleStateProvider(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      DiscreteValueRuleStateProvider
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  GetState(...)
 |      GetState(self: maliput.api.rules.DiscreteValueRuleStateProvider, id: maliput.api.rules.Rule.Id) -> Optional[maliput::api::rules::StateProviderResult<maliput::api::rules::DiscreteValueRule::DiscreteValue>]
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data and other attributes defined here:
 |  
 |  StateResult = <class 'maliput.api.rules.DiscreteValueRuleStateProvider...
 |  
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from pybind11_b

In [58]:
lane_s_range = maliput.api.LaneSRange(p_road.lane.id(), maliput.api.SRange(0, p_road.lane.length()))
rules = rulebook.FindRules([lane_s_range], 0.01)
help(rules)
help(rules.discrete_value_rules)
print(list(rules.discrete_value_rules.keys()))
print(list(rules.discrete_value_rules.values()))

Help on QueryResults in module maliput.api.rules object:

class QueryResults(pybind11_builtins.pybind11_object)
 |  Method resolution order:
 |      QueryResults
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, /, *args, **kwargs)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  direction_usage
 |  
 |  discrete_value_rules
 |  
 |  range_value_rules
 |  
 |  right_of_way
 |  
 |  speed_limit
 |  
 |  ----------------------------------------------------------------------
 |  Static methods inherited from pybind11_builtins.pybind11_object:
 |  
 |  __new__(*args, **kwargs) from pybind11_builtins.pybind11_type
 |      Create and return a new object.  See help(type) for accurate signature.

Help on dict object:

class dict(object)
 |  dict() -> new empty dictionary
 |  dict(

In [76]:
# Iterating discrete value
for rule_id in rules.discrete_value_rules:
    rule = rules.discrete_value_rules[rule_id]
    if rule.type_id() == direction_usage:
        state_result = discrete_provider.GetState(rule.id())
        state = state_result.state
        help(state)
        print("Rule {}. State {}. Severity {}".format(rule.id().string(), state.value, state.severity))
        

Help on DiscreteValue in module maliput.api.rules object:

class DiscreteValue(State)
 |  Method resolution order:
 |      DiscreteValue
 |      State
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(...)
 |      __eq__(self: maliput.api.rules.DiscreteValueRule.DiscreteValue, arg0: maliput.api.rules.DiscreteValueRule.DiscreteValue) -> bool
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: maliput.api.rules.DiscreteValueRule.DiscreteValue) -> None
 |      
 |      2. __init__(self: maliput.api.rules.DiscreteValueRule.DiscreteValue, severity: int, related_rules: Dict[str, List[maliput.api.rules.Rule.Id]], related_unique_ids: Dict[str, List[maliput.api.UniqueId]], value: str) -> None
 |  
 |  __ne__(...)
 |      __ne__(self: maliput.api.rules.DiscreteValueRule.DiscreteValue, arg0: maliput.api.rules.DiscreteValueRule.DiscreteValue) -> bool
 |  
 |  --------

In [79]:
# Iterating range value
for rule_id in rules.range_value_rules:
    rule = rules.range_value_rules[rule_id]
    if rule.type_id() == speed_limit:
        state_result = range_provider.GetState(rule.id())
        state = state_result.state
        help(state)
        print("Rule {}. Min {}. Max {}. Severity {}".format(rule.id().string(), state.min, state.max, state.severity))
        

Help on Range in module maliput.api.rules object:

class Range(State)
 |  Method resolution order:
 |      Range
 |      State
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __eq__(...)
 |      __eq__(self: maliput.api.rules.RangeValueRule.Range, arg0: maliput.api.rules.RangeValueRule.Range) -> bool
 |  
 |  __init__(...)
 |      __init__(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. __init__(self: maliput.api.rules.RangeValueRule.Range) -> None
 |      
 |      2. __init__(self: maliput.api.rules.RangeValueRule.Range, severity: int, related_rules: Dict[str, List[maliput.api.rules.Rule.Id]], related_unique_ids: Dict[str, List[maliput.api.UniqueId]], description: str, min: float, max: float) -> None
 |  
 |  __lt__(...)
 |      __lt__(self: maliput.api.rules.RangeValueRule.Range, arg0: maliput.api.rules.RangeValueRule.Range) -> bool
 |  
 |  __ne__(...)
 |      __ne__(self: maliput.api.rules.RangeValueRule.Ra

In [ ]:
# A compact example of getting the direction, vehicle usage, and speed for a lane.
